In [1]:
!pip install -q graphframes

In [2]:
import pyspark
from graphframes import *
from pyspark.sql.functions import desc, col

In [3]:
# Put your `generate_graph` function here
def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
    Returns:
    --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
    '''
    edges = pages.flatMap(lambda x: set([(x[0], link[0]) for link in x[1]]))
    list_vertices = edges.map(lambda x: ((x[1], 0)))
    groupi = edges.groupByKey()
    keys_vertices = groupi.map(lambda x: (x[0], 0))
    vertices = (keys_vertices + list_vertices).groupByKey().map(lambda x: [x[0]])

    return edges, vertices

In [ ]:
# t_start = time()
bucket_name = "elad_318640828_anchor_bucket"
pages_links = spark.read.parquet(f'gs://elad_318640828/Parquet_Files').select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
# pr_time = time() - t_start
pr.show()

NameError: name 'col' is not defined